**Recitation 0: Checkpointing**

We will show you how to checkpoint and load your model :D

**Section 0: Setup**

Let's define a quick dummy model, optimizer, and scheduler that we'll be saving and loading :)

In [1]:
!pip install wandb --quiet # Install WandB

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

os.environ['WANDB_API_KEY'] = ""#your key here
wandb.login()

In [3]:
# A simple submodule
class DummySubmodule(nn.Module):
    def __init__(self):
        super(DummySubmodule, self).__init__()
        self.layer = nn.Linear(in_features = 32, out_features = 32)

    def forward(self, x):
        return self.layer(x)

# A simple network
class DummyNetwork(nn.Module):

    def __init__(self):
        super(DummyNetwork, self).__init__()

        self.lower_layer = nn.Sequential(
            nn.Linear(in_features = 32, out_features = 64),
            nn.ReLU(),
            nn.Linear(in_features = 32, out_features = 64),
            nn.ReLU(),
        )
        self.upper_layer = nn.Sequential(
            nn.Linear(in_features = 64, out_features = 32),
            nn.ReLU(),
        )

        self.module1 = DummySubmodule()

    def forward(self, x):
        res = self.lower_layer(x)
        res = self.upper_layer(res)
        res = self.submodule(res)
        return res

# Declare the model, optimizer, and scheduler
model = DummyNetwork().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

Let's take a look at some of the information we can checkpoint

In [4]:
# Print model's state_dict
print("==============================================================")
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
print("==============================================================")
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
print("==============================================================")
# Print scheduler's state_dict
print("\nScheduler's state_dict:")
for var_name, value in scheduler.state_dict().items():
    print(var_name, "\t", value)
print("==============================================================")

Model's state_dict:
lower_layer.0.weight 	 torch.Size([64, 32])
lower_layer.0.bias 	 torch.Size([64])
lower_layer.2.weight 	 torch.Size([64, 32])
lower_layer.2.bias 	 torch.Size([64])
upper_layer.0.weight 	 torch.Size([32, 64])
upper_layer.0.bias 	 torch.Size([32])
module1.layer.weight 	 torch.Size([32, 32])
module1.layer.bias 	 torch.Size([32])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'initial_lr': 0.01, 'params': [0, 1, 2, 3, 4, 5, 6, 7]}]

Scheduler's state_dict:
step_size 	 10
gamma 	 0.1
base_lrs 	 [0.01]
last_epoch 	 0
_step_count 	 1
_is_initial 	 False
_get_lr_called_within_step 	 False
_last_lr 	 [0.01]


#######################################################################################################

**Section 1: How to save the checkpoint Saving a checkpoint**

Checkpointing locally

In [5]:
# let's pretend we're in the middle of our training
epoch = 6 # pretend we're in our 6th epoch
loss = 0.78 #pretend this is our model's loss at the moment

checkpoint_path=f"<run_name>_{epoch}.pth"

# Saving your states locally with torch.save
torch.save({
    'model_state_dict': model.state_dict(),   # saving the model state
    # if isinstance(model, nn.DataParallel) 'model_state_dict': model.module.state_dict()
    'optimizer_state_dict': optimizer.state_dict(),   # saving the optimizer state
    'scheduler_state_dict': scheduler.state_dict(),   # saving the scheduler state
    'epoch': epoch,
    'current_loss': loss
    }, checkpoint_path
)



Alternative way of Saving Model Checkpoints Using Helper Function(an example from HW)


In [ ]:
# Alternative approach: Helper function method (commonly used in HW assignments)
# This provides a cleaner, reusable way to save all checkpoint components
def save_model(model, optimizer, scheduler, metrics, epoch, path):
    """
    Helper function to save model checkpoint locally.
    
    Args:
        model: The neural network model
        optimizer: The optimizer
        scheduler: The learning rate scheduler
        metrics: Training metrics (e.g., loss, accuracy)
        epoch: Current epoch number
        path: File path to save the checkpoint
    """
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         'metric'                   : metrics,
         'epoch'                    : epoch},
         path)
    print(f"Checkpoint saved to {path}")

# Example usage:
metrics = {'loss': 0.78, 'accuracy': 0.92}
save_model(model, optimizer, scheduler, metrics, epoch=6, path="checkpoint_epoch6.pth")

Checkpointing and saving to wandb as an artifact

In [ ]:
# Before the run, you need to have started a run like so....
run = wandb.init(
    project="wandb-quickstart",
    name="<run_name>",
    )

# ...
# ...
# ...
# Within a training loop (or wherever else you want)....

# Option 1:
# create artifacts (keeps track of versioning, and is much more organized to work with between collaborators)
checkpoint_artifact = wandb.Artifact("<run_name>", type="checkpoint") # You can switch type="model if you only want to save a model"

checkpoint_artifact.add_file(checkpoint_path)

run.log_artifact(checkpoint_artifact)

# Option 2:
# directly save the model to wandb
wandb.save(checkpoint_path, base_path=os.path.dirname(checkpoint_path))

#######################################################################################################

**Section 2\: Loading a checkpoint file into our current model**

Downloading a model from wandb

In [ ]:
# METHOD 1: Download from wandb Artifact
# If you need to re-obtain the run, you can do the following....
api = wandb.Api()
# information can be obtained from the wandb link adddress as follows:
# https://wandb.ai/<USERNAME>/<PROJECT_NAME>/runs/<RUN_ID>?nw=nwuser<USERNAME>
run = api.run("<USERNAME>/<PROJECT_NAME>/<RUN_ID>")

# To retrieve the artifact....
# Get the artifact (choose which version of the model you want)
artifact = run.use_artifact('<run_name>:latest')
# Downloading the artifact
artifact_dir = artifact.download()
# Loading the model dict
checkpoint_dict = torch.load(os.path.join(artifact_dir, '<run_name>'))


# METHOD 2: Download the directly saved file from wandb to Local File
checkpoint_file = wandb.restore('<run_name>', run_path="<USERNAME>/<PROJECT_NAME>/<RUN_ID>").name
checkpoint_dict = torch.load(checkpoint_file)

Loading a .pth checkpoint file from our local directory to our model

In [ ]:
# .pth checkpoint file path can also be obtained from a locally saved .pth file. Or, you can use the checkpoint_dict obtained from the prior wandb artifact download :)
checkpoint_path = "/content/<run_name>_6.pth"
checkpoint_dict = torch.load(checkpoint_path)


# Loading model weights
# if isinstance(model, nn.DataParallel) model.module.load_state_dict(checkpoint_dict['model_state_dict'])
model.load_state_dict(checkpoint_dict['model_state_dict'])
# Loading optimizer state
optimizer.load_state_dict(checkpoint_dict['optimizer_state_dict'])
# Loading the scheduler state
scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
# find the epoch we left off at
current_epoch = checkpoint_dict['epoch']
# Find any metrics that might be relevant
current_loss = checkpoint_dict['current_loss']

# Done!!!!!!

<ipython-input-7-2bf5796c687f>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint_path)


Alternative way of Loading Model Checkpoints Using Helper Function(an example from HW)

In [ ]:

# Alternative approach: Helper function method (commonly used in HW assignments)
# This provides a cleaner, reusable way to load checkpoints with optional components
def load_model(model, optimizer=None, scheduler=None, path='./checkpoint.pth'):
    """
    Helper function to load model checkpoint from local storage.
    
    Args:
        model: The neural network model to load weights into
        optimizer: The optimizer (optional, can be None for inference)
        scheduler: The learning rate scheduler (optional, can be None)
        path: File path of the checkpoint to load
    
    Returns:
        model, optimizer, scheduler, epoch, metrics
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        optimizer = None
    
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    else:
        scheduler = None
    
    epoch = checkpoint['epoch']
    metrics = checkpoint['metric']
    
    print(f"Checkpoint loaded from {path}")
    print(f"Resuming from epoch {epoch} with metrics: {metrics}")
    
    return model, optimizer, scheduler, epoch, metrics

# Example usage 1: Load for resuming training (with optimizer and scheduler)
model, optimizer, scheduler, epoch, metrics = load_model(
    model, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    path="checkpoint_epoch6.pth"
)

# Example usage 2: Load for inference only (no optimizer or scheduler needed)
model, _, _, epoch, metrics = load_model(
    model, 
    optimizer=None, 
    scheduler=None, 
    path="checkpoint_epoch6.pth"
)

In [ ]:
# If you want to load specific parts of your model (in our case, we can load just the lower layers or just the upper layers)
specific_weights = { # Creates dictionary of only desired weights
    key: value
    for key, value in checkpoint_dict['model_state_dict'].items()
    if 'lower_layer' in key
}

model.load_state_dict(specific_weights, strict=False)